In [2]:
"""
Example of an arbitrary task and how to wrap it up with a GridSearchTask.

In this example, the task generates its own data. So the score method just
calls fit method and returns the performance metric.

In this case, train/test data are generated separately. 
But they could be organized arbitrarily differently
as long as the score method then knows how to return the 
performance of the network on the task (with the given esn_params).
"""
from echoes import ESNPredictive
from echoes.model_selection import GridSearchTask


class NewTask:
    def __init__(self, a=1, b=2, esn_params=None):
        self.a = a
        self.b = b
        self.esn_params = esn_params
    
    def train_data_maker(self):
        """Function that generates train data for your taks"""
        return None
    
    def test_data_maker(self):
        """Function that generates test data for your taks"""
        return None
    
    def fit(self):
        # Do whatever you want, eg train en ESN to remember patterns.
        # For example, something that looks like this        
        self.esn = ESNPredictive(**self.esn_params)
        X_train, y_train = self.train_data_maker()
        self.esn.fit(X_train, y_train) 
    
    def score(self):        
        self.fit()
        X_test, y_test = self.test_data_maker()
        y_predicted = self.esn.predict(X_test)        
        return mean_squared_error(y_predicted, y_test)

    
# So the NewTask class can be easily wrapped up by GridSearchTask

param_grid = {}  # esn parameters to test, eg. bias, spectral radius
task_param = {}  # task parameters, eg. time lags, noise, these are fixed

grid = GridSearchTask(NewTask, task_params, param_grid)
grid.fit()
grid.to_dataframe()   # export results to dataframe
grid.to_dataframe().to_csv("filename")   # export results to file